In [1]:
%pip install openai flask
%pip install PyPDF2 python-docx
%pip install transformers datasets
%pip install openai
%pip install gpt_index
%pip install langchain
%pip install gradio
%pip install python-pptx
%pip install torch
%pip install transformers
%pip install docx2txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
from flask import Flask, request, jsonify
import PyPDF2
import docx

In [3]:
OPENAI_API_KEY = ''
openai.api_key = OPENAI_API_KEY


In [13]:
def chatgpt_query(prompt):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )

    answer = response.choices[0].text.strip()
    return answer

def pdf_to_text(file_path):
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfFileReader(f)
        text = ''
        for page_num in range(reader.numPages):
            text += reader.getPage(page_num).extractText()
    return text

def docx_to_text(file_path):
    doc = docx.Document(file_path)
    text = '\n'.join([para.text for para in doc.paragraphs])
    return text

def convert_documents_to_text(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        output_file_name = os.path.splitext(file_name)[0] + '.txt'
        output_file_path = os.path.join(output_folder, output_file_name)

        if file_name.lower().endswith('.pdf'):
            text = pdf_to_text(file_path)
        elif file_name.lower().endswith('.docx'):
            text = docx_to_text(file_path)
        else:
            continue

        with open(output_file_path, 'w', encoding='utf-8') as f:
            f.write(text)

input_folder = 'docs/'
output_folder = 'gpt_it_files/'
convert_documents_to_text(input_folder, output_folder)



In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the data (assuming data is already collected and cleaned)
data = [
    ("user_question_1", "agent_response_1"),
    ("user_question_2", "agent_response_2"),
    # ...
]

# Tokenize and pad the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text for pair in data for text in pair])

questions, answers = zip(*data)
questions_seq = tokenizer.texts_to_sequences(questions)
answers_seq = tokenizer.texts_to_sequences(answers)

max_len = max([len(seq) for seq in questions_seq + answers_seq])
questions_seq = pad_sequences(questions_seq, maxlen=max_len, padding="post")
answers_seq = pad_sequences(answers_seq, maxlen=max_len, padding="post")

# Split the data into training and validation sets
split_idx = int(0.8 * len(questions_seq))
train_questions, val_questions = questions_seq[:split_idx], questions_seq[split_idx:]
train_answers, val_answers = answers_seq[:split_idx], answers_seq[split_idx:]

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the data (assuming data is already collected and cleaned)
data = [
    ("user_question_1", "agent_response_1"),
    ("user_question_2", "agent_response_2"),
    # ...
]

# Tokenize and pad the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text for pair in data for text in pair])

questions, answers = zip(*data)
questions_seq = tokenizer.texts_to_sequences(questions)
answers_seq = tokenizer.texts_to_sequences(answers)

max_len = max([len(seq) for seq in questions_seq + answers_seq])
questions_seq = pad_sequences(questions_seq, maxlen=max_len, padding="post")
answers_seq = pad_sequences(answers_seq, maxlen=max_len, padding="post")

# Split the data into training and validation sets
split_idx = int(0.8 * len(questions_seq))
train_questions, val_questions = questions_seq[:split_idx], questions_seq[split_idx:]
train_answers, val_answers = answers_seq[:split_idx], answers_seq[split_idx:]

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# Set parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128
lstm_units = 256

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([train_questions, train_answers[:,:-1]], train_answers[:, 1:, np.newaxis],
          validation_data=([val_questions, val_answers[:,:-1]], val_answers[:, 1:, np.newaxis]),
          epochs=50, batch_size=64)